In [15]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage import color, exposure, transform
from skimage.exposure import equalize_hist
import cv2
import os
from scipy import fftpack
from skimage.transform import warp
from skimage import io
from image_registration import chi2_shift
import skimage
import image_registration
import tifffile
import time
import concurrent.futures
import skimage
import image_registration

In [16]:
def generateCoefficents():
    refrences = []
    base = "/home/humangene/Desktop/background/BG"
    refrences.append(cv2.imread(base+'_cR.tif',cv2.IMREAD_UNCHANGED))
    refrences.append(cv2.imread(base+'_cG.tif',cv2.IMREAD_UNCHANGED))
    refrences.append(cv2.imread(base+'_cB.tif',cv2.IMREAD_UNCHANGED))
   
    refrenceMean = 50000
    means = []
    coeficients = []

    for index, image in enumerate(refrences):
        coeficients.append([])
        means.append([])
        for line in range(image.shape[1]):
            if(np.mean(image[:, line:line+1]) != 0):
                coeficients[index].append(refrenceMean/ np.mean(image[:, line]))
            else:
                coeficients[index].append(1)
            means[index].append(np.mean(image[:, line]) )
    
    return coeficients

def shift_finder(R,G):
    height, width = R.shape
    orb_detector = cv2.ORB_create(10000)
    kp1, d1 = orb_detector.detectAndCompute(R, None)
    kp2, d2 = orb_detector.detectAndCompute(G, None)
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    matches = matcher.match(d1, d2)
    matches = sorted(matches, key = lambda x:x.distance)
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt

    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)
    return homography

def align_channel(chanel,M):
    a = skimage.transform.SimilarityTransform(matrix=M)
    return np.ma.array(warp(chanel, a, preserve_range=True,output_shape=chanel.shape, cval=-1)) 

In [17]:
base_folder = "../IR_RED_filter/"
num_lines = len(os.listdir(base_folder))//3
coeficients = generateCoefficents()
lines = {}
i = num_lines//2
R = cv2.imread(base_folder+"line_"+str(i)+"_c1.tif",cv2.IMREAD_GRAYSCALE) *coeficients[0]
G = cv2.imread(base_folder+"line_"+str(i)+"_c2.tif",cv2.IMREAD_GRAYSCALE)*coeficients[1]
B = cv2.imread(base_folder+"line_"+str(i)+"_c3.tif",cv2.IMREAD_GRAYSCALE)*coeficients[2]
BR_h = shift_finder(R.astype("uint8"),B.astype("uint8"))
GR_h = shift_finder(R.astype("uint8"),G.astype("uint8"))

In [18]:
images = {}
img_nums = list(range(num_lines))
def reading_images(img_num):
    print(str(img_num)+",")
    images[img_num] =(
                        np.clip(
                                cv2.merge([
                                            (cv2.imread(base_folder+"line_"+str(img_num)+"_c1.tif",cv2.IMREAD_GRAYSCALE)/255)*coeficients[0],
                                            (align_channel(cv2.imread(base_folder+"line_"+str(img_num)+"_c2.tif",cv2.IMREAD_GRAYSCALE),GR_h)/255)*coeficients[1],
                                            (align_channel(cv2.imread(base_folder+"line_"+str(img_num)+"_c3.tif",cv2.IMREAD_GRAYSCALE),BR_h)/255)*coeficients[2]
                                            ])[2:-6,2:-15]
                                ,0,1)*255
                    ).astype("uint8")
    return img_num
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(reading_images,img_nums)
concurrent.futures.as_completed(results)
t2 = time.perf_counter()
print(t2-t1)

0,
1,
2,
3,4,

5,
6,
7.715546305989847


In [19]:
line_shifts = {}
img_nums = list(range(num_lines-1))

def process_image(img_num):
    print(img_num)
    right_image = images[img_num]
    left_image = images[img_num+1]
    min_row = min(right_image.shape[0],left_image.shape[0])
    min_col = min(right_image.shape[1],left_image.shape[1])
    right_image_c = right_image[:min_row,:300]
    left_image_c = left_image[:min_row,-300:]
    im1 = cv2.cvtColor(left_image_c, cv2.COLOR_BGR2GRAY)
    im2 = cv2.cvtColor(right_image_c, cv2.COLOR_BGR2GRAY)
    yoff,xoff ,yer,xer= image_registration.chi2_shift(im1,im2,0.1)
    im1_aligned_to_im2ed_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)
    print(yoff,xoff)
    if yoff > 30  and xoff<0:  
        line_shifts[(i,i+1)] = (int(yoff),int(-1*xoff))
t1 = time.perf_counter()
with concurrent.futures.ProcessPoolExecutor(max_workers = num_lines-1) as executor:
    results = list(executor.map(process_image,img_nums))
concurrent.futures.as_completed(results)
t2 = time.perf_counter()
print(t2-t1)

0
1
2
3
4
5


/opt/anaconda/lib/python3.9/site-packages/image_registration/fft_tools/convolve_nd.py:269: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bigarray[arrayslices] = array
/opt/anaconda/lib/python3.9/site-packages/image_registration/fft_tools/convolve_nd.py:269: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  bigarray[arrayslices] = array
/opt/anaconda/lib/python3.9/site-packages/image_registration/fft_tools/convolve_nd.py:270: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of

54.115234375 -15.130859375
39.509765625 36.623046875-16.126953125 
-15.318359375
51.400390625 -14.462890625
45.138671875 -14.955078125
43.130859375 -15.447265625
4.758348013041541


In [20]:
valid_list = []
for key , value in line_shifts.items():
    valid_list.append(key[1])
    
yoff_sum = 0
xoff_sum = 0
tempelate = images[3]
max_tot_row = tempelate.shape[0]
max_tot_col = tempelate.shape[1]
for val in line_shifts.values():
    yoff_sum+=val[0]
    xoff_sum+=val[1]
img_shape = (max_tot_row,max_tot_col,3)
mid = (int(img_shape[0]/2),int(img_shape[1]/2))
row_numbers = img_shape[0]
tot = np.zeros((row_numbers+xoff_sum,img_shape[1]*num_lines - yoff_sum,3),dtype = np.uint8)
last_row_index = tot.shape[0]
last_col_index = tot.shape[1] 
valid_range = [last_row_index,last_row_index]
for i in range(num_lines):
    indx = num_lines - i-1
    print(indx)
    if i in valid_list:
        middle_img = images[i]
        img_shape = (middle_img.shape[0],middle_img.shape[1],3)
        row_numbers = middle_img.shape[0]
        yshift,xshift = line_shifts[(i - 1,i)]
        tot[last_row_index-row_numbers - xshift:last_row_index-xshift,last_col_index-img_shape[1]+yshift:last_col_index+yshift,:] = middle_img
        valid_range[0] = last_row_index-row_numbers
        valid_range[1] = last_row_index+xshift
        last_row_index = last_row_index-xshift
        last_col_index = last_col_index-img_shape[1]+yshift
    else:
        middle_img = images[i]
        tot[last_row_index-middle_img.shape[0]:last_row_index,last_col_index-middle_img.shape[1]:last_col_index,:] = middle_img
        valid_range[0] = last_row_index-middle_img.shape[0]
        valid_range[1] = last_row_index
        
        last_row_index = last_row_index
        last_col_index = last_col_index - middle_img.shape[1]

6
5
4
3
2
1
0


In [21]:
tifffile.imwrite("./tot_multi.png",tot)